In [1]:
import torch, torch.nn.functional as F, torch.nn as nn
from model import custom_res18
from dataset import full_forget_retain_loader_train, full_forget_retain_loader_test
from train_test_acc import train_model, model_test
from mia_unlearning_score import UnLearningScore, get_membership_attack_prob, actv_dist
from seed import set_seed
import gc

In [2]:
set_seed(42)
BATCH = 512
FORGET_RATIO = 0.1
DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
retrain = custom_res18(10).to(DEVICE)
retrain.load_state_dict(torch.load('./retrain_tr_model.pth', map_location='cpu'))
retrain.eval()
full_tr, combined_loader, forget_tr, retain_tr = full_forget_retain_loader_train(FORGET_RATIO, batch_size=BATCH)
full_te, combined_loader, forget_te, retain_te = full_forget_retain_loader_test(FORGET_RATIO, batch_size=BATCH)


/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/root/anaconda3/envs/khw/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Files already downloaded and verified
Files already downloaded and verified


In [3]:
print('\n[Accuracies]')
model_test(retrain, retain_te.dataset, 'retrain/Retain_test', batch_size=256, device=DEVICE)
model_test(retrain, forget_te.dataset, 'retrain/Forget_test', batch_size=256, device=DEVICE)
model_test(retrain, retain_tr.dataset, 'retrain/Retain_train', batch_size=256, device=DEVICE)
model_test(retrain, forget_tr.dataset, 'retrain/Forget_train', batch_size=256, device=DEVICE)

print('\n[MIA]')
forget_eval_test  = torch.utils.data.DataLoader(forget_te.dataset,  batch_size=128, shuffle=False)
retain_eval_train = torch.utils.data.DataLoader(retain_tr.dataset,  batch_size=128, shuffle=False)
retain_eval_test  = torch.utils.data.DataLoader(retain_te.dataset,  batch_size=128, shuffle=False)
mia_p = get_membership_attack_prob(retain_eval_train, forget_eval_test, retain_eval_test, retrain, DEVICE)
print(f'  MIA success prob on Forget (test) : {mia_p:.4f}')


[Accuracies]
[Test:retrain/Retain_test] Acc 80.03%
[Test:retrain/Forget_test] Acc 77.90%
[Test:retrain/Retain_train] Acc 99.84%
[Test:retrain/Forget_train] Acc 81.16%

[MIA]
  MIA success prob on Forget (test) : 0.7052
